# Pattern Driven Stock Analysis

Finding Similar Stocks using Graph Analytis in Neo4j.


# Steps

1. Import Stock Data into Neo4j
2. Create [:NEXT] relationship property between nodes
3. Find KNN similarity using trade volume among nodes: Create [:SIMILAR_KNN_VOLUME]  relationship among PRICE Nodes
4. Community detection based on Louvain algorithm using [:SIMILAR_KNN_VOLUME] relationship: Create VolumeCommunity property
5. Hierarchical Structures Communities using Label Propagation Algorithm

In [ ]:
! pip install python-dotenv ipython-cypher neo4j py2neo yfiles_jupyter_graphs graphdatascience pandas --quiet

In [ ]:
import os
%load_ext dotenv
%dotenv

PATIENT_DATA_PATH=os.getenv("PATIENT_DATA_PATH")
NEO4J_HOST=os.getenv("NEO4J_HOST")
NEO4J_USER=os.getenv("NEO4J_USER")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

### Import Stock Data into Neo4j

Using NASDAQ Data from: https://www.kaggle.com/datasets/kalilurrahman/nasdaq100-stock-price-data

In [ ]:
q = """
LOAD CSV WITH HEADERS FROM 'http://fileserver:4858/patient_data/csv/payers.csv' AS row
MERGE (p:Payer {id:row.Id})
          SET p.name=row.NAME,
            p.address=row.ADDRESS,
            p.city=row.CITY,
            p.zip=row.ZIP,
            p.state=row.STATE_HEADQUARTERED
"""
cypher_run(q)

Create [:NEXT] relationship property between nodes

In [ ]:
  # Create NEXT relationship between encounters of a patient
q = """
    CALL apoc.periodic.iterate(
    'MATCH (p:Patient) RETURN p',
    'MATCH (p)-[:HAS_ENCOUNTER]->(e)
    WITH e
    ORDER BY e.date
    WITH collect(e) AS encounters
    WITH encounters, encounters[1..] as nextEncounters
    UNWIND range(0,size(nextEncounters)-1,1) as index
    WITH encounters[index] as first, nextEncounters[index] as second
    CREATE (first)-[:NEXT]->(second)',{iterateList:false});
    
    """
cypher_run(q)

### KNN
Find KNN similarity using trade volume among nodes, using [:SIMILAR_KNN_VOLUME] relationship among PRICE Nodes

In [ ]:
query = """

CALL gds.nodeSimilarity.write.estimate('northwindGraph1', {
  writeRelationshipType: 'SIMILAR',
  writeProperty: 'score'
})
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory

"""

In [ ]:
query = """
CALL gds.nodeSimilarity.stream('northwindGraph1')
YIELD node1, node2, similarity
RETURN gds.util.asNode(node1).customerID AS customer1, gds.util.asNode(node2).customerID AS customer2, similarity
ORDER BY similarity DESCENDING, customer1, customer2
"""

cypher_run(query)

### Community detection
Using Louvain algorithm on [:SIMILAR_KNN_VOLUME] relationship to Create VolumeCommunity property

In [3]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/

### Label Propagation Algorithm
Hierarchical Structures Communities using

In [2]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/label-propagation/